# importando tabula e arquivos pdf
- importante que os PDFs estejam todos na mesma página dos arquivos

In [6]:
import tabula
import pandas as pd

lista_suap = tabula.read_pdf("suap/2023_02_RMA_suap_com_ajustes.pdf", pages="1", encoding='Ansi')
mes = '2023_02_ajustes_inventario'
# lista em dataframe
df_suap = lista_suap[0]

# removendo linhas com valores nulos
df_suap = df_suap.dropna()
display(df_suap)

df_suap = df_suap[:-1]

# removendo e renomeando colunas
df_suap = df_suap.drop(columns=['Nome', 'Estoque', 'Entrada'])
df_suap['ED'] = df_suap['Cod.']
df_suap['saida'] = df_suap['Saída']
df_suap['saldo_suap'] = df_suap['Estoque.1']
df_suap = df_suap.drop(columns=['Cod.', 'Estoque.1', 'Saída'])

# substituindo caracteres para a conversão em numero
df_suap['saldo_suap'] = df_suap['saldo_suap'].str.replace(".", "", regex=True)
df_suap['saldo_suap'] = df_suap['saldo_suap'].str.replace(",", ".", regex=True)
df_suap['saida'] = df_suap['saida'].str.replace(".", "", regex=True)
df_suap['saida'] = df_suap['saida'].str.replace(",", ".", regex=True)


df_suap['saldo_suap'] = pd.to_numeric(df_suap['saldo_suap'])
df_suap['saida'] = pd.to_numeric(df_suap['saida'])
print("{:.2f}".format(df_suap['saldo_suap'].sum()))
print(df_suap.info())
display(df_suap)

,Cod.,Nome,Estoque,Entrada,Saída,Estoque.1
1,07,GENEROS DE ALIMENTACAO,"884,00","0,00","884,00","0,00"
2,14,MATERIAL EDUCATIVO E ESPORTIVO,"40.159,38","0,00","5.989,36","34.170,02"
3,16,MATERIAL DE EXPEDIENTE,"62.865,09","0,00","7.920,06","54.945,03"
4,17,MATERIAL DE PROCESSAMENTO DE DADOS,"16.356,19","0,00","2.758,20","13.597,99"
5,19,MATERIAL DE ACONDICIONAMENTO E,"2.332,87","0,00","12,00","2.320,87"
7,21,MATERIAL DE COPA E COZINHA,"7.290,60","0,00","671,19","6.619,41"
8,22,MATERIAL DE LIMPEZA E PROD. DE,"24.184,48","0,00","6.774,40","17.410,08"
10,23,"UNIFORMES, TECIDOS E AVIAMENTOS","16.512,14","0,00","6.654,95","9.857,19"
11,24,MATERIAL P/ MANUT.DE BENS,"42.195,67","0,00","10.230,47","31.965,20"
13,25,MATERIAL P/ MANUTENCAO DE BENS,"226,96","0,00","27,36","199,60"


223127.72
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 1 to 22
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ED          17 non-null     object 
 1   saida       17 non-null     float64
 2   saldo_suap  17 non-null     float64
dtypes: float64(2), object(1)
memory usage: 544.0+ bytes
None


,ED,saida,saldo_suap
1,07,884.00,0.00
2,14,5989.36,34170.02
3,16,7920.06,54945.03
4,17,2758.20,13597.99
5,19,12.00,2320.87
7,21,671.19,6619.41
8,22,6774.40,17410.08
10,23,6654.95,9857.19
11,24,10230.47,31965.20
13,25,27.36,199.60


In [7]:
# parâmetros lattice=True e guess=False para conseguir reconhecer e montar tabelas em arquivos com formatação diferente

lista_siafi = tabula.read_pdf("siafi/2023_02_RMA_siafi_sem_ajustes.pdf", guess=False, pages="1-2")

tabela1 = lista_siafi[0]
tabela1.columns = tabela1.iloc[7]
tabela1 = tabela1[8:20]

tabela2 = lista_siafi[1]
tabela2.columns = tabela2.iloc[7]
tabela2 = tabela2[8:15]

# Renomeando coluna por causa do caracter especial $

tabela1.columns = ['conta']
tabela2.columns = ['conta']

# unindo tabelas pela coluna conta
df_siafi = pd.merge(tabela1, tabela2, how = 'outer', on = 'conta')

# fatiando coluna para obter colunas ED e Saldo
df_siafi['ED'] = df_siafi['conta'].str[:4]
df_siafi['saldo_siafi'] = df_siafi['conta'].str[4:-1]

# removendo coluna ajustada
df_siafi = df_siafi.drop(columns=['conta'])

# substituindo caracteres para a conversão em numero
df_siafi['saldo_siafi'] = df_siafi['saldo_siafi'].str.replace(".", "", regex=True)
df_siafi['saldo_siafi'] = df_siafi['saldo_siafi'].str.replace(",", ".", regex=True)
df_siafi['ED'] = df_siafi['ED'].str.replace("P ", "", regex=True)
df_siafi = df_siafi.drop(index=[17,18])

df_siafi = df_siafi.drop(index=16)
df_siafi['saldo_siafi'] = pd.to_numeric(df_siafi['saldo_siafi'])

print("{:.2f}".format(df_siafi['saldo_siafi'].sum()))
display(df_siafi)
print(df_siafi.info())
# display(df_siafi)

279756.83


,ED,saldo_siafi
0,14,40159.38
1,16,62865.09
2,17,16356.19
3,19,2332.87
4,21,7290.60
5,22,24184.48
6,23,16512.14
7,24,42195.67
8,25,226.96
9,26,29867.13


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ED           16 non-null     object 
 1   saldo_siafi  16 non-null     float64
dtypes: float64(1), object(1)
memory usage: 384.0+ bytes
None


In [8]:
# unindo data frames do suap e siafi em uma unica tabela
df_rma = pd.merge(df_suap, df_siafi, how = 'inner', on='ED')
df_rma = df_rma.reindex(columns=['ED','saldo_suap', 'saida', 'saldo_siafi'])

# calculando a diferença entre colunas 
df_rma['diferenca'] = df_rma['saldo_suap'] - df_rma['saldo_siafi']

# definindo a ação para cafa resultado
acao = []
for valor in df_rma['diferenca']:
    
    if valor > 0:
        baixa = "baixa no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "baixa no siafi"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

df_rma['acao'] = acao

display(df_rma)
df_rma.to_clipboard()
df_rma.to_excel(f'saida/{mes}.xlsx')

,ED,saldo_suap,saida,saldo_siafi,diferenca,acao
0,14,34170.02,5989.36,40159.38,-5989.36,baixa no siafi
1,16,54945.03,7920.06,62865.09,-7920.06,baixa no siafi
2,17,13597.99,2758.20,16356.19,-2758.20,baixa no siafi
3,19,2320.87,12.00,2332.87,-12.00,baixa no siafi
4,21,6619.41,671.19,7290.60,-671.19,baixa no siafi
5,22,17410.08,6774.40,24184.48,-6774.40,baixa no siafi
6,23,9857.19,6654.95,16512.14,-6654.95,baixa no siafi
7,24,31965.20,10230.47,42195.67,-10230.47,baixa no siafi
8,25,199.60,27.36,226.96,-27.36,baixa no siafi
9,26,19485.81,10381.32,29867.13,-10381.32,baixa no siafi
